# Example 4 - Spatial variation of Curie depth

In Example 2 and 3 we computed the Curie depth for a single point using a fixed window. Here, compute the Curie depth over a magnetic anomaly and estimate its uncertainty.

### Contents

- [Optimisation routine](#Optimisation-routine)
- [Uncertainty analysis](#Uncertainty-analysis)
- [Sensitivity analysis](#Sensitivity-analysis)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# initialise CurieOptimise object
grid = pycurious.CurieOptimise(d, xmin, xmax, ymin, ymax)

## Optimisation routine

Here we iteratively evaluate the Curie depth across the magnetic anomaly. We use gradient-based inversion which is deterministic, because it doesn't include uncertainty, but a quick means to recover Curie depth from the magnetic anomaly.

In [ ]:
# get centroids

window_size = 200e3
xc_list, yc_list = grid.create_centroid_list(window_size, spacingX=10e3, spacingY=10e3)

print("number of centroids = {}".format(len(xc_list)))

In [ ]:
# no priors
grid.reset_priors()

beta, zt, dz, C = grid.optimise_routine(window_size, xc_list, yc_list)

In [ ]:
# get dimensions of domain
xcoords = np.unique(xc_list)
ycoords = np.unique(yc_list)
nc, nr = xcoords.size, ycoords.size


# plot results
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(17,3.))

im1 = ax1.imshow(beta.reshape(nr,nc))
im2 = ax2.imshow(zt.reshape(nr,nc))
im3 = ax3.imshow(dz.reshape(nr,nc))
im4 = ax4.imshow(C.reshape(nr,nc))

fig.colorbar(im1, ax=ax1, label=r"$\beta$")
fig.colorbar(im2, ax=ax2, label=r"$z_t$")
fig.colorbar(im3, ax=ax3, label=r"$\Delta z$")
fig.colorbar(im4, ax=ax4, label=r"$C$")

In [ ]:
# plot Curie depth

curie_depth = zt + dz

fig = plt.figure()
ax1 = fig.add_subplot(111)
im1 = ax1.imshow(curie_depth.reshape(nr,nc), cmap=plt.cm.BrBG)
fig.colorbar(im1)

## Uncertainty analysis

The variation of Curie depth is not particularly useful without any estimate of the uncertainty. We conduct the same MCMC simulation we performed in the previous example [Ex3-Posing-the-inverse-problem](./Ex3-Posing-the-inverse-problem.ipynb) to sample the posterior.

In [ ]:
x_scale = [0.25, 0.1, 1.0, 0.5]

# run more simulations for production runs
burnin = 1000
nsim = 5000

# mean across the domain
mu_beta, mu_zt, mu_dz, mu_C = beta.mean(), zt.mean(), dz.mean(), C.mean()

pt_post = []

# This will take some time
for xc, yc in zip(xc_list, yc_list):
    posterior = grid.metropolis_hastings(window_size, xc, yc, nsim, burnin, x_scale,\
                                         mu_beta, mu_zt, mu_dz, mu_C, taper=None)
    pt_post.append( posterior )

In [ ]:
curie_depth = np.zeros_like(xc_list)
uncertainty = np.zeros_like(xc_list)

for i, pt in enumerate(pt_post):
    betaP, ztP, dzP, CP = pt
    cpd = ztP + dzP
    curie_depth[i] = np.mean(cpd)
    uncertainty[i] = np.std(cpd)

In [ ]:
# plot Curie depth

curie_depth = zt + dz

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(11,4))
im1 = ax1.imshow(curie_depth.reshape(nr,nc), cmap=plt.cm.BrBG)
im2 = ax2.imshow(uncertainty.reshape(nr,nc), cmap=plt.cm.Blues)
fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax2)

ax1.set_title("Curie depth (km)")
ax2.set_title("Uncertainty (km)")

## Sensitivity analysis

The sensitivity analysis is a gradient-based solution to approximate the posterior distribution (read the motivation for this in Tarantola: *Inverse Problem Theory*). The radial power spectrum $\Phi$ is randomly perturbed within its uncertainty $\sigma_{\Phi}$ to sample locally around the MAP estimate.

$$
\Phi_{d}^* = \Phi_d + \delta \Phi_d
$$

where $\delta \Phi_d$ is a random perturbation generated from the probability density of $\Phi_d$. This is equivalent to pertubing the likelihood function,

$$
P(\Phi|\mathbf{m}) = \frac{1}{(\sqrt{2\pi} \sigma_\Phi)^n} \exp \left( - \frac{\lvert \Phi(\mathbf{m}) - \Phi_d^* \lvert^2}{2 \sigma_\Phi^2} \right)
$$

where $\mathbf{m}^*$ is a perturbed prior distribution: $\mathbf{m}_p^* = \mathbf{m}_p + \delta \mathbf{m}_p$. If the prior is a Gaussian normal distribution, then the perturbed prior is

$$
P(\mathbf{m}) = \frac{1}{\sqrt{2\pi} \sigma_p} \exp \left( - \frac{\lvert \mathbf{m} - \mathbf{m}_p^* \lvert^2}{2 \sigma_p^2} \right)
$$

Prior distributions can be added by

```python
from scipy import stats
beta_p = stats.norm(3.0, 1.0)
grid.add_prior(beta=beta_p)
```

We repeat the inversion multiple times, sampling different values from $P(\mathbf{m})$ and $P(\Phi|\mathbf{m})$ to evaluate the uncertainty in each variable.

In [ ]:
nsim = 100
pt_post = []

for xc, yc in zip(xc_list, yc_list):
    sensitivity = grid.sensitivity(window_size, xc, yc, nsim, mu_beta, mu_zt, mu_dz, mu_C, taper=None)
    pt_post.append( sensitivity )

In [ ]:
curie_depth = np.zeros_like(xc_list)
uncertainty = np.zeros_like(xc_list)

for i, pt in enumerate(pt_post):
    betaP, ztP, dzP, CP = pt
    cpd = ztP + dzP
    curie_depth[i] = np.mean(cpd)
    uncertainty[i] = np.std(cpd)

In [ ]:
# plot Curie depth

curie_depth = zt + dz

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(11,4))
im1 = ax1.imshow(curie_depth.reshape(nr,nc), cmap=plt.cm.BrBG)
im2 = ax2.imshow(uncertainty.reshape(nr,nc), cmap=plt.cm.Blues)
fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax2)

ax1.set_title("Curie depth (km)")
ax2.set_title("Uncertainty (km)")